In [1]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.python.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import nltk
import re

Using TensorFlow backend.


In [93]:
df = pd.read_csv("/Users/Itua/Downloads/boe_articles2015.csv",encoding='"cp1252"')


In [94]:
df['article_text'][0]

'The Bank of England has made great progress towards a single, unified institution that can exploit synergies across our policy functions. The Prudential Regulation Authority is already seeing the benefits of regulating banks and insurers in concert with monetary and macroeconomic policymaking. The One Bank plan set out an ambitious vision for an institution that could best discharge the enormous new powers and responsibilities bestowed on it following the global financial crisis, including the responsibility for prudential regulation of banks and insurers, the design and operation of macroprudential policy, and the conduct of monetary policy. Our objective is a unified, diverse, and talented institution, that delivers excellent analysis and outstanding execution, and that is transparent, professional and accountable to the public and to Parliament. We have achieved a lot in the last year in shaping the Bank of the future with the PRA at its heart. We are making our judgement-based app

In [95]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [96]:
sentences[:1]


['The Bank of England has made great progress towards a single, unified institution that can exploit synergies across our policy functions.']

In [61]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [97]:
len(word_embeddings)


400000

In [98]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [99]:
#nltk.download('stopwords')

In [100]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


In [101]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [102]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [103]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [104]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [105]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [106]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [107]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [108]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)


In [109]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])


The One Bank plan set out an ambitious vision for an institution that could best discharge the enormous new powers and responsibilities bestowed on it following the global financial crisis, including the responsibility for prudential regulation of banks and insurers, the design and operation of macroprudential policy, and the conduct of monetary policy.
This project combined the Bank’s assessment of the main domestic risks to UK financial stability with portfolio-level assessments undertaken by PRA specialists, to deliver an assessment of the strength of our largest banks individually as well as a view of the resilience of the banking system as a whole.
The Bank of England has made great progress towards a single, unified institution that can exploit synergies across our policy functions.
The Bank of England has made great progress towards a single, unified institution that can exploit synergies across our policy functions.
We have harmonised our reward, compensation and terms and cond